<a href="https://colab.research.google.com/github/dnyanda12/assignments/blob/main/GIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import geemap.geemap as map
import ee
ee.Authenticate()
ee.Initialize(project='ee-dnyandagawathe24')
m = map.Map()

In [13]:
Kerala = ee.FeatureCollection('projects/ee-dnyandagawathe24/assets/KERALA_SHP')
m.addLayer(Kerala,{},'Kerala')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
region = ee.Geometry.Polygon(
     [[[73.6345947,13.0302592],
     [77.8612510,13.1870040],
     [77.8612510,7.7350127],
     [73.6345947,7.7350127],
     [73.6345947,13.0302592]]],

    None,
    False)

In [14]:
NDVI = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')

In [15]:
NDVI_KA = NDVI.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = NDVI_KA.filterDate('2023-02-18', '2024-02-18');

In [16]:
filter = ee.Filter.equals(leftField ='doy',rightField = 'doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY,NDVI_KA,filter))

In [17]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [18]:
vis_para = {
  'min': 0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [20]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDVI_mean'],**vis_para).clip(Kerala))

In [21]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 10,
  'format':'gif',
}

In [22]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-dnyandagawathe24/videoThumbnails/6a69937a93a40ecef16b17a0396eb23e-ea1b1588c710372ebe68aa0726285b33:getPixels
